## PoseBusters Benchmark Structure Prediction Chemical Similarity Analysis

#### Import packages

In [ ]:
import itertools
import json
import os
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import DataStructs
from tqdm import tqdm

#### Configure packages

In [ ]:
pd.options.mode.copy_on_write = True

#### Define constants

In [ ]:
# General variables
baseline_methods = [
    # "vina_p2rank",
    # "diffdock",
    # "dynamicbind",
    # "rfaa",
    # "alphafold3",
    # "chai-lab",
    # "neuralplexer",
    # "flowdock_hp",
    # "flowdock_aft",
    # "flowdock_esmfold",
    "flowdock",
]
max_num_repeats_per_method = 3

# Filepaths for each baseline method
globals()["vina_output_dir"] = os.path.join("..", "forks", "Vina", "inference")
globals()["diffdock_output_dir"] = os.path.join("..", "forks", "DiffDock", "inference")
globals()["dynamicbind_output_dir"] = os.path.join(
    "..", "forks", "DynamicBind", "inference", "outputs", "results"
)
globals()["rfaa_output_dir"] = os.path.join("..", "forks", "RoseTTAFold-All-Atom", "inference")
globals()["alphafold3_output_dir"] = os.path.join("..", "forks", "alphafold3", "inference")
globals()["chai-lab_output_dir"] = os.path.join("..", "forks", "chai-lab", "inference")
globals()["neuralplexer_output_dir"] = os.path.join("..", "forks", "NeuralPLexer", "inference")
globals()["flowdock_hp_output_dir"] = os.path.join("..", "forks", "FlowDock", "hp_inference")
globals()["flowdock_aft_output_dir"] = os.path.join("..", "forks", "FlowDock", "aft_inference")
globals()["flowdock_esmfold_output_dir"] = os.path.join(
    "..", "forks", "FlowDock", "esmfold_inference"
)
globals()["flowdock_output_dir"] = os.path.join("..", "forks", "FlowDock", "inference")

for repeat_index in range(1, max_num_repeats_per_method + 1):
    # P2Rank-Vina results
    globals()[f"vina_p2rank_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["vina_output_dir"],
        f"vina_p2rank_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"vina_p2rank_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["vina_output_dir"],
            f"vina_p2rank_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # DiffDock results
    globals()[f"diffdock_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["diffdock_output_dir"],
        f"diffdock_posebusters_benchmark_output_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"diffdock_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["diffdock_output_dir"],
            f"diffdock_posebusters_benchmark_output_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # DynamicBind results
    globals()[f"dynamicbind_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["dynamicbind_output_dir"],
        f"posebusters_benchmark_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"dynamicbind_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["dynamicbind_output_dir"],
            f"posebusters_benchmark_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # RoseTTAFold-All-Atom results
    globals()[f"rfaa_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["rfaa_output_dir"],
        f"rfaa_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"rfaa_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["rfaa_output_dir"],
        f"rfaa_posebusters_benchmark_outputs_{repeat_index}_relaxed",
        "bust_results.csv",
    )

    # AlphaFold 3 (Single-Seq) results
    globals()[f"alphafold3_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["alphafold3_output_dir"],
        f"alphafold3_ss_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"alphafold3_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["alphafold3_output_dir"],
            f"alphafold3_ss_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # Chai-1 (Single-Seq) results
    globals()[f"chai-lab_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["chai-lab_output_dir"],
        f"chai-lab_ss_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"chai-lab_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["chai-lab_output_dir"],
            f"chai-lab_ss_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # NeuralPLexer results
    globals()[f"neuralplexer_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["neuralplexer_output_dir"],
        f"neuralplexer_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"neuralplexer_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["neuralplexer_output_dir"],
            f"neuralplexer_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # FlowDock-HP results
    globals()[f"flowdock_hp_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["flowdock_hp_output_dir"],
        f"flowdock_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"flowdock_hp_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["flowdock_hp_output_dir"],
            f"flowdock_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # FlowDock-AFT results
    globals()[f"flowdock_aft_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["flowdock_aft_output_dir"],
        f"flowdock_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"flowdock_aft_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["flowdock_aft_output_dir"],
            f"flowdock_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # FlowDock-ESMFold results
    globals()[f"flowdock_esmfold_posebusters_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["flowdock_esmfold_output_dir"],
            f"flowdock_posebusters_benchmark_outputs_{repeat_index}",
            "bust_results.csv",
        )
    )
    globals()[f"flowdock_esmfold_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["flowdock_esmfold_output_dir"],
            f"flowdock_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

    # FlowDock results
    globals()[f"flowdock_posebusters_bust_results_csv_filepath_{repeat_index}"] = os.path.join(
        globals()["flowdock_output_dir"],
        f"flowdock_posebusters_benchmark_outputs_{repeat_index}",
        "bust_results.csv",
    )
    globals()[f"flowdock_posebusters_relaxed_bust_results_csv_filepath_{repeat_index}"] = (
        os.path.join(
            globals()["flowdock_output_dir"],
            f"flowdock_posebusters_benchmark_outputs_{repeat_index}_relaxed",
            "bust_results.csv",
        )
    )

# Mappings
method_mapping = {
    "vina_p2rank": "P2Rank-Vina",
    "diffdock": "DiffDock-L",
    "dynamicbind": "DynamicBind",
    "rfaa": "RoseTTAFold-AA",
    "alphafold3": "AF3-Single-Seq",
    "chai-lab": "Chai-1-Single-Seq",
    "neuralplexer": "NeuralPLexer",
    "flowdock_hp": "FlowDock-HP",
    "flowdock_aft": "FlowDock-AFT",
    "flowdock_esmfold": "FlowDock-ESMFold",
    "flowdock": "FlowDock-AF3",
}

method_category_mapping = {
    "vina_p2rank": "Conventional blind",
    "diffdock": "DL-based blind",
    "dynamicbind": "DL-based blind",
    "rfaa": "DL-based blind",
    "alphafold3": "DL-based blind",
    "chai-lab": "DL-based blind",
    "neuralplexer": "DL-based blind",
    "flowdock_hp": "DL-based blind",
    "flowdock_aft": "DL-based blind",
    "flowdock_esmfold": "DL-based blind",
    "flowdock": "DL-based blind",
}

# Metrics
BUST_TEST_COLUMNS = [
    # accuracy #
    "rmsd_≤_2å",
    # chemical validity and consistency #
    "mol_pred_loaded",
    "mol_true_loaded",
    "mol_cond_loaded",
    "sanitization",
    "molecular_formula",
    "molecular_bonds",
    "tetrahedral_chirality",
    "double_bond_stereochemistry",
    # intramolecular validity #
    "bond_lengths",
    "bond_angles",
    "internal_steric_clash",
    "aromatic_ring_flatness",
    "double_bond_flatness",
    "internal_energy",
    # intermolecular validity #
    "minimum_distance_to_protein",
    "minimum_distance_to_organic_cofactors",
    "minimum_distance_to_inorganic_cofactors",
    "volume_overlap_with_protein",
    "volume_overlap_with_organic_cofactors",
    "volume_overlap_with_inorganic_cofactors",
]

#### Report test results for each baseline method

In [ ]:
# load and report test results for each baseline method
for config in ["", "_relaxed"]:
    for method in baseline_methods:
        for repeat_index in range(1, max_num_repeats_per_method + 1):
            method_title = method_mapping[method]

            if not os.path.exists(
                globals()[f"{method}_posebusters{config}_bust_results_csv_filepath_{repeat_index}"]
            ):
                continue

            globals()[f"{method}_posebusters{config}_bust_results_{repeat_index}"] = pd.read_csv(
                globals()[f"{method}_posebusters{config}_bust_results_csv_filepath_{repeat_index}"]
            )
            assert (
                "mol_id"
                in globals()[f"{method}_posebusters{config}_bust_results_{repeat_index}"].columns
            ), f"mol_id column not found in {method_title}{config} bust_results CSV file for repeat {repeat_index}"
            globals()[
                f"{method}_posebusters{config}_bust_results_table_{repeat_index}"
            ] = globals()[f"{method}_posebusters{config}_bust_results_{repeat_index}"][
                BUST_TEST_COLUMNS + ["rmsd", "mol_id", "inchi_crystal"]
            ]
            globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"].loc[
                :, "pb_valid"
            ] = (
                globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"]
                .iloc[:, 1:-3]
                .all(axis=1)
            )

            globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"].loc[
                :, "method"
            ] = method
            globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"].loc[
                :, "post-processing"
            ] = ("energy minimization" if config == "_relaxed" else "none")
            globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"].loc[
                :, "dataset"
            ] = "posebusters"
            globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"].loc[
                :, "docked_ligand_successfully_loaded"
            ] = globals()[f"{method}_posebusters{config}_bust_results_table_{repeat_index}"][
                ["mol_pred_loaded", "mol_true_loaded", "mol_cond_loaded"]
            ].all(
                axis=1
            )

            globals()[f"{method}{config}_bust_results_table_{repeat_index}"] = globals()[
                f"{method}_posebusters{config}_bust_results_table_{repeat_index}"
            ]

            print(
                f"{method_title}{config}_{repeat_index} PoseBusters Benchmark set `rmsd_≤_2å`: {globals()[f'{method}_posebusters{config}_bust_results_table_{repeat_index}']['rmsd_≤_2å'].mean()}"
            )
            print(
                f"{method_title}{config}_{repeat_index} PoseBusters Benchmark set `rmsd_≤_2å and pb_valid`: {globals()[f'{method}_posebusters{config}_bust_results_table_{repeat_index}'][globals()[f'{method}_posebusters{config}_bust_results_table_{repeat_index}']['pb_valid']]['rmsd_≤_2å'].sum() / len(globals()[f'{method}_posebusters{config}_bust_results_table_{repeat_index}'])}"
            )

#### Define helper functions

In [ ]:
def assign_method_index(method: str) -> str:
    """
    Assign method index for plotting.

    :param method: Method name.
    :return: Method index.
    """
    return list(method_mapping.keys()).index(method)


def categorize_method(method: str) -> str:
    """
    Categorize method for plotting.

    :param method: Method name.
    :return: Method category.
    """
    return method_category_mapping.get(method, "Misc")


def get_uniprot_mapping(pdb_id: str) -> set:
    """
    Get the UniProt mapping for a given PDB ID using the PDBe API.

    :param pdb_id: PDB ID (e.g., "1A2B").
    :return: Set of UniProt IDs mapped to the PDB ID, or None if not found.
    """
    # use the PDBe API to get the mapping data
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id.lower()}"
    response = requests.get(url)
    if response.status_code != 200:
        return None
    data = response.json()
    uniprot_ids = set()
    if pdb_id.lower() in data:
        # NOTE: each mapping corresponds to one chain/segment mapped to a UniProt entry
        for uniprot_id in data[pdb_id.lower()].get("UniProt", []):
            uniprot_ids.add(uniprot_id)
    return uniprot_ids


def group_has_low_similarity(group: pd.DataFrame, threshold: float = 0.6) -> bool:
    """
    Return True if every pair of molecules in the group has Tanimoto similarity less than threshold.
    Groups with only one ligand are considered valid.

    :param group: DataFrame group containing the fingerprints.
    :param threshold: Similarity threshold for filtering.
    :return: True if the group has low similarity, False otherwise.
    """
    max_sim = 0.0
    fps = group["fp"].tolist()
    if len(fps) < 2:
        group["max_fp_sim"] = max_sim
        return True
    for fp1, fp2 in itertools.combinations(fps, 2):
        # Only calculate similarity if both fingerprints are valid.
        if fp1 is not None and fp2 is not None:
            sim = DataStructs.FingerprintSimilarity(fp1, fp2)
            max_sim = max(max_sim, sim)
            if sim >= threshold:
                return False
    group["max_fp_sim"] = max_sim
    return True

#### Standardize metrics

In [ ]:
# load and organize the PoseBusters Benchmark results CSVs
for repeat_index in range(1, max_num_repeats_per_method + 1):
    globals()[f"results_table_{repeat_index}"] = pd.concat(
        [
            globals()[f"{method}{config}_bust_results_table_{repeat_index}"]
            for method in baseline_methods
            for config in ["", "_relaxed"]
            if f"{method}{config}_bust_results_table_{repeat_index}" in globals()
        ]
    )
    globals()[f"results_table_{repeat_index}"].loc[:, "method_category"] = globals()[
        f"results_table_{repeat_index}"
    ]["method"].apply(categorize_method)
    globals()[f"results_table_{repeat_index}"].loc[:, "method_assignment_index"] = globals()[
        f"results_table_{repeat_index}"
    ]["method"].apply(assign_method_index)
    globals()[f"results_table_{repeat_index}"].loc[:, "rmsd_within_threshold"] = (
        globals()[f"results_table_{repeat_index}"].loc[:, "rmsd_≤_2å"].fillna(False)
    )
    globals()[f"results_table_{repeat_index}"].loc[:, "rmsd_within_threshold_and_pb_valid"] = (
        globals()[f"results_table_{repeat_index}"].loc[:, "rmsd_within_threshold"]
    ) & (globals()[f"results_table_{repeat_index}"].loc[:, "pb_valid"].fillna(False))
    globals()[f"results_table_{repeat_index}"].loc[:, "RMSD ≤ 2 Å & PB-Valid"] = (
        globals()[f"results_table_{repeat_index}"]
        .loc[:, "rmsd_within_threshold_and_pb_valid"]
        .astype(int)
    )
    globals()[f"results_table_{repeat_index}"].loc[:, "RMSD ≤ 2 Å"] = (
        globals()[f"results_table_{repeat_index}"]
        .loc[:, "rmsd_within_threshold"]
        .fillna(False)
        .astype(int)
    )
    globals()[f"results_table_{repeat_index}"].loc[:, "dataset"] = (
        globals()[f"results_table_{repeat_index}"]
        .loc[:, "dataset"]
        .map({"posebusters": "PoseBusters Benchmark set"})
    )
    globals()[f"results_table_{repeat_index}"].loc[:, "method"] = (
        globals()[f"results_table_{repeat_index}"].loc[:, "method"].map(method_mapping)
    )

    # group ligands according to their binding proteins (here defined as their first UniProt IDs, if available)
    mol_to_protein_mapping = dict()
    protein_grouped_mols = defaultdict(list)
    mol_ids = list(set(globals()[f"results_table_{repeat_index}"].mol_id))

    protein_grouped_mols_json_filepath = (
        f"posebusters_benchmark_protein_grouped_mols_{repeat_index}.json"
    )
    mol_to_protein_mapping_json_filepath = (
        f"posebusters_benchmark_mol_to_protein_mapping_{repeat_index}.json"
    )

    if os.path.exists(protein_grouped_mols_json_filepath):
        # load cached groupings if available
        with open(protein_grouped_mols_json_filepath, "r") as f:
            protein_grouped_mols = json.load(f)
    if os.path.exists(mol_to_protein_mapping_json_filepath):
        # load cached mappings if available
        with open(mol_to_protein_mapping_json_filepath, "r") as f:
            mol_to_protein_mapping = json.load(f)

    if not (protein_grouped_mols and mol_to_protein_mapping):
        for mol in tqdm(mol_ids, desc="Grouping ligands by protein"):
            pdb_id = mol.split("_")[0]
            uniprot_ids = get_uniprot_mapping(pdb_id)

            group_uniprot_id = None

            if uniprot_ids:
                for uniprot_id in uniprot_ids:
                    if uniprot_id in protein_grouped_mols:
                        group_uniprot_id = uniprot_id
                        protein_grouped_mols[group_uniprot_id].append(mol)
                        break

                if not group_uniprot_id:
                    group_uniprot_id = list(sorted(uniprot_ids))[0]
                    protein_grouped_mols[group_uniprot_id] = [mol]
            else:
                group_uniprot_id = "Unknown"

            mol_to_protein_mapping[mol] = group_uniprot_id

        # cache groupings
        with open(protein_grouped_mols_json_filepath, "w") as f:
            json.dump(protein_grouped_mols, f)
        with open(mol_to_protein_mapping_json_filepath, "w") as f:
            json.dump(mol_to_protein_mapping, f)

    # add the protein group IDs to the results table
    globals()[f"results_table_{repeat_index}"].loc[:, "protein_id"] = globals()[
        f"results_table_{repeat_index}"
    ]["mol_id"].map(mol_to_protein_mapping)

#### Make plots

In [ ]:
# RMSD Violin Plot of PoseBusters Benchmark Set Results (Relaxed vs. Unrelaxed and Grouped Per Protein) #

# prepare data for the violin plots to plot
colors = ["#FB8072", "#BEBADA"]

# combine results for each dataset across all three repeats
combined_data_list = []
for repeat_index in range(1, max_num_repeats_per_method + 1):
    ad_relaxed_results_table = globals()[f"results_table_{repeat_index}"][
        (globals()[f"results_table_{repeat_index}"]["dataset"] == "PoseBusters Benchmark set")
        & (globals()[f"results_table_{repeat_index}"]["post-processing"] == "energy minimization")
        & (globals()[f"results_table_{repeat_index}"]["protein_id"] != "Unknown")
    ]
    ad_relaxed_results_table.loc[
        :, "protein_group_size"
    ] = ad_relaxed_results_table.protein_id.value_counts()[
        ad_relaxed_results_table.protein_id
    ].values
    ad_relaxed_results_table = ad_relaxed_results_table[
        ad_relaxed_results_table.protein_group_size > 1
    ].reset_index()  # filter out singleton protein groups

    ad_relaxed_results_table["mol"] = ad_relaxed_results_table["inchi_crystal"].apply(
        lambda x: Chem.MolFromInchi(x)
    )
    ad_relaxed_results_table["fp"] = ad_relaxed_results_table["mol"].apply(
        lambda m: Chem.RDKFingerprint(m) if m is not None else None
    )
    ad_relaxed_results_table = pd.concat(
        [
            group
            for _, group in ad_relaxed_results_table.groupby("protein_id")
            if group_has_low_similarity(group, threshold=0.6)
        ]
    )

    ad_unrelaxed_results_table = globals()[f"results_table_{repeat_index}"][
        (globals()[f"results_table_{repeat_index}"]["dataset"] == "PoseBusters Benchmark set")
        & (globals()[f"results_table_{repeat_index}"]["post-processing"] == "none")
        & (globals()[f"results_table_{repeat_index}"]["protein_id"] != "Unknown")
    ]
    ad_unrelaxed_results_table.loc[
        :, "protein_group_size"
    ] = ad_unrelaxed_results_table.protein_id.value_counts()[
        ad_unrelaxed_results_table.protein_id
    ].values
    ad_unrelaxed_results_table = ad_unrelaxed_results_table[
        ad_unrelaxed_results_table.protein_group_size > 1
    ].reset_index()  # filter out singleton protein groups

    ad_unrelaxed_results_table["mol"] = ad_unrelaxed_results_table["inchi_crystal"].apply(
        lambda x: Chem.MolFromInchi(x)
    )
    ad_unrelaxed_results_table["fp"] = ad_unrelaxed_results_table["mol"].apply(
        lambda m: Chem.RDKFingerprint(m) if m is not None else None
    )
    ad_unrelaxed_results_table = pd.concat(
        [
            group
            for _, group in ad_unrelaxed_results_table.groupby("protein_id")
            if group_has_low_similarity(group, threshold=0.6)
        ]
    )

    combined_data_list.append(pd.concat([ad_relaxed_results_table, ad_unrelaxed_results_table]))

combined_relaxed_data = (
    pd.concat(combined_data_list).sort_values("method_assignment_index").reset_index(drop=True)
)

assert (
    len(combined_relaxed_data.method.unique()) == 1
), f"Expected only one method in the combined data, but found: {combined_relaxed_data.method.unique()}"

# define font properties
plt.rcParams["font.size"] = 12
plt.rcParams["axes.labelsize"] = 14

# set the size of the figure
plt.figure(figsize=(12, 6))

# create a violin plot grouped by protein_id
sns.violinplot(
    x="protein_id",
    y="rmsd",
    hue="post-processing",
    # ignore outliers for better readability
    data=combined_relaxed_data[combined_relaxed_data["rmsd"] < 10],
    split=True,
    inner="quartile",
    palette=colors,
    cut=0,
)

# set labels and title
plt.xlabel("UniProt ID")
plt.ylabel("RMSD")

# rotate x-axis labels for better readability
plt.xticks(rotation=45, ha="right")

# display legend outside the plot
plt.legend(title="Post-processing", bbox_to_anchor=(1.05, 1), loc="best")

# display the plots
plt.tight_layout()
plt.savefig(
    "posebusters_benchmark_structure_prediction_relaxed_rmsd_violin_plot_by_protein.png", dpi=300
)
plt.show()